In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime

# === Configuration ===
DATA_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\FSLFacialExpressionsLanguages"
os.makedirs(DATA_DIR, exist_ok=True)

# Ask user for label name
label = input("Effort").strip().lower()
SAVE_PATH = os.path.join(DATA_DIR, f"{label}.csv")

# === Landmark indices (reduce redundancy) ===
SELECTED_FACE_IDXS = [
    33, 133, 362, 263,   # eyes
    1, 2, 4, 5, 45, 275, # nose bridge + cheeks
    61, 291, 0, 17, 57, 287  # mouth + chin
]

# === MediaPipe Setup ===
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)
face_mesh = mp_face.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.8
)

# === Smoothing helpers ===
def smooth_landmarks(landmarks, prev_landmarks, alpha=0.5):
    """Apply exponential moving average to reduce jitter."""
    if prev_landmarks is None:
        return landmarks
    return alpha * np.array(landmarks) + (1 - alpha) * np.array(prev_landmarks)

# === Dataset parameters ===
MAX_SAMPLES = 600       # capture limit per label
MIN_HAND_CONF = 0.75
FRAME_SKIP = 2          # process every 2nd frame to reduce duplicates
SMOOTHING = True

# === Camera setup ===
cap = cv2.VideoCapture(0)
frame_count = 0
saved_count = 0
data = []
prev_landmarks = None

print(f"📷 Starting data capture for '{label}' in 3 seconds...")
time.sleep(3)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_count += 1

    # Skip frames to reduce redundancy
    if frame_count % FRAME_SKIP != 0:
        continue

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb)
    face_results = face_mesh.process(rgb)

    row = []
    hands_detected = False
    face_detected = False

    # === HAND LANDMARKS (80%) ===
    hand_points = []
    if hand_results.multi_hand_landmarks:
        hands_detected = True
        for hand_landmarks in hand_results.multi_hand_landmarks[:2]:
            for lm in hand_landmarks.landmark:
                hand_points.extend([lm.x, lm.y, lm.z])
        if len(hand_results.multi_hand_landmarks) == 1:
            hand_points.extend([0] * (21 * 3))
    else:
        hand_points.extend([0] * (21 * 3 * 2))

    # === FACE LANDMARKS (20%) ===
    face_points = []
    if face_results.multi_face_landmarks:
        face_detected = True
        face_landmarks = face_results.multi_face_landmarks[0]
        for idx in SELECTED_FACE_IDXS:
            lm = face_landmarks.landmark[idx]
            face_points.extend([lm.x, lm.y, lm.z])
    else:
        face_points.extend([0] * (len(SELECTED_FACE_IDXS) * 3))

    # === Combine all features ===
    full_landmarks = hand_points + face_points

    # Smooth landmarks to reduce noise
    if SMOOTHING:
        full_landmarks = smooth_landmarks(full_landmarks, prev_landmarks)
    prev_landmarks = full_landmarks

    # === Append metadata ===
    row.extend(full_landmarks)
    row.append(label)
    row.append(datetime.utcnow().isoformat())

    # Save only if valid hand detected
    if hands_detected or face_detected:
        data.append(row)
        saved_count += 1
        status = f"✔ Collected ({saved_count}/{MAX_SAMPLES})"
    else:
        status = "❌ No hands or face detected"

    # Stop once enough samples are collected
    if saved_count >= MAX_SAMPLES:
        print("\n✅ Capture complete.")
        break

    # === Draw Landmarks ===
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            mp_draw.draw_landmarks(
                frame, face_landmarks, mp_face.FACEMESH_TESSELATION,
                mp_draw.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                mp_draw.DrawingSpec(color=(0, 0, 255), thickness=1)
            )

    # === HUD ===
    cv2.putText(frame, f"Label: {label}", (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame, f"Samples: {saved_count}/{MAX_SAMPLES}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(frame, status, (10, 105),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0) if hands_detected else (0,0,255), 2)
    cv2.imshow("FSL Capture (Improved v2)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\n🛑 Manual stop triggered.")
        break

cap.release()
cv2.destroyAllWindows()

# === Save CSV ===
columns = []
for hand in ["L1_", "L2_"]:
    for i in range(21):
        columns += [f"{hand}x{i}", f"{hand}y{i}", f"{hand}z{i}"]
for idx in SELECTED_FACE_IDXS:
    columns += [f"Fx{idx}", f"Fy{idx}", f"Fz{idx}"]
columns += ["label", "timestamp"]

df = pd.DataFrame(data, columns=columns)

if not df.empty:
    df.to_csv(SAVE_PATH, index=False)
    print(f"\n💾 Dataset saved: {SAVE_PATH}")
    print(f"📊 Total samples: {len(df)} | Features per sample: {len(columns) - 2}")
else:
    print("🚫 No valid samples collected.")


In [ ]:
import os
import pandas as pd

# === Configuration ===
DATASET_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\FSLFacialExpressionsLanguages\Expressions"
combined_path = os.path.join(DATASET_DIR, "FSL_Facial_NMG_dataset.csv")

# === Scan for CSV files ===
csv_files = [f for f in os.listdir(DATASET_DIR) if f.endswith(".csv")]
print(f"📁 Found {len(csv_files)} dataset files")

all_dfs = []
all_columns = set()

# === First pass: Collect all columns ===
for csv_file in csv_files:
    path = os.path.join(DATASET_DIR, csv_file)
    try:
        df = pd.read_csv(path)
        if df.empty:
            print(f"⚠️ Skipped empty file: {csv_file}")
            continue
        if "label" not in df.columns:
            print(f"⚠️ 'label' missing in {csv_file}, skipping...")
            continue
        all_columns.update(df.columns)
        print(f"✅ Columns collected from: {csv_file} ({len(df)} samples)")
    except Exception as e:
        print(f"❌ Error reading {csv_file}: {e}")

if not all_columns:
    raise SystemExit("🚫 No valid CSV files found.")

# === Enforce column order identical to capture script ===
SELECTED_FACE_IDXS = [
    33,133,362,263,   # eyes
    1,2,4,5,45,275,   # nose + cheeks
    61,291,0,17,57,287 # mouth + chin
]

columns = []
for hand in ["L1_", "L2_"]:
    for i in range(21):
        columns += [f"{hand}x{i}", f"{hand}y{i}", f"{hand}z{i}"]
for idx in SELECTED_FACE_IDXS:
    columns += [f"Fx{idx}", f"Fy{idx}", f"Fz{idx}"]
columns += ["label", "timestamp"]

# Use this explicit order instead of sorted(all_columns)
all_columns = columns
print(f"✅ Using fixed column order from capture script ({len(all_columns)} columns)")

# === Second pass: Align and merge ===
for csv_file in csv_files:
    path = os.path.join(DATASET_DIR, csv_file)
    try:
        df = pd.read_csv(path)
        if df.empty or "label" not in df.columns:
            continue
        df["label"] = df["label"].astype(str).str.strip().str.lower()
        df = df.reindex(columns=all_columns, fill_value=0)
        all_dfs.append(df)
        print(f"✅ Loaded {csv_file} ({len(df)} samples)")
    except Exception as e:
        print(f"❌ Error loading {csv_file}: {e}")

# === Merge all datasets ===
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)
    final_df.to_csv(combined_path, index=False)

    print("\n✅ Combined dataset created successfully!")
    print(f"📄 Saved to: {combined_path}")
    print("🧮 Total samples:", final_df.shape[0])

    if "label" in final_df.columns:
        print("\n📊 Samples per label:")
        print(final_df["label"].value_counts())
        print("\n🏷️ Unique labels:", list(final_df["label"].unique()))
    else:
        print("⚠️ 'label' column missing in merged file!")
else:
    print("🚫 No valid datasets found to merge.")


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import joblib

# === Configuration ===
DATA_PATH = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\FSLFacialExpressionsLanguages\FSL_Facial_NMG_dataset.csv"
MODEL_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\TensorFlow\FSL-FacialNonManualGrammarLanguages"
MODEL_PATH = os.path.join(MODEL_DIR, "nmg_model_tf.keras")
TFLITE_PATH = os.path.join(MODEL_DIR, "nmg_model_tf.tflite")

os.makedirs(MODEL_DIR, exist_ok=True)

# === Step 1: Load and Inspect Dataset ===
print("📥 Loading dataset...")
df = pd.read_csv(DATA_PATH)
print(f"✅ Dataset loaded: {df.shape[0]} samples, {df.shape[1]} columns")

if "label" not in df.columns:
    raise ValueError("🚫 'label' column not found in dataset!")

# === Step 2: Clean Data ===
X = df.drop(columns=["label", "timestamp"], errors="ignore").fillna(0)
y = df["label"]
X = X.select_dtypes(include=[np.number])
# Replace zeros with NaN and fill with column means to stabilize scaling
X = X.replace(0, np.nan)
X = X.fillna(X.mean())
print(f"🔧 Replaced zeros with feature means → shape: {X.shape}")



print(f"🔢 Feature matrix shape: {X.shape}")
print("🏷️ Labels found:", y.unique())

# === Step 3: Balance Classes ===
print("⚖️ Balancing dataset...")
max_samples = y.value_counts().max()
df_balanced = df.groupby("label", group_keys=False).apply(
    lambda x: resample(x, replace=True, n_samples=max_samples, random_state=42)
)
X = df_balanced.drop(columns=["label", "timestamp"], errors="ignore").fillna(0)
y = df_balanced["label"]
print(df_balanced["label"].value_counts())

# === Step 4: Encode Labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
print(f"🔢 Encoded {num_classes} classes →", list(label_encoder.classes_))

# === Step 5: Normalize Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Step 6: Split Data ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)
print("📊 Data split complete (80% train / 20% test)")

# === Step 7: Define Neural Network Model ===
input_dim = X_train.shape[1]

model = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  # ✅ compatible version
    metrics=['accuracy']
)


model.summary()

# === Step 8: Train Model with EarlyStopping ===
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_accuracy', verbose=1)
]

print("\n🚀 Training model...")
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,          # ⬅️ longer training
    batch_size=16,       # ⬅️ smaller batch for finer updates
    callbacks=callbacks,
    verbose=1
)

# === Step 9: Print Accuracy Summary per Epoch ===
print("\n📈 Training Accuracy Progress:")
for epoch, (train_acc, val_acc) in enumerate(zip(history.history['accuracy'], history.history['val_accuracy']), 1):
    print(f"Epoch {epoch:03d} → Train: {train_acc:.4f} | Val: {val_acc:.4f}")

# === Step 10: Final Accuracy Summary ===
train_acc_final = history.history['accuracy'][-1] * 100
val_acc_final = history.history['val_accuracy'][-1] * 100
print(f"\n✅ Final Training Accuracy: {train_acc_final:.2f}%")
print(f"✅ Final Validation Accuracy: {val_acc_final:.2f}%")


# === Step 11: Evaluate Model ===
y_pred = np.argmax(model.predict(X_test), axis=1)
acc = accuracy_score(y_test, y_pred)

print(f"\n🎯 Test Accuracy: {acc * 100:.2f}%")
print("\n📄 Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("🧾 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# === Step 12: Save Assets ===
joblib.dump(scaler, os.path.join(MODEL_DIR, "nmg_model_tf_scaler.pkl"))
np.save(os.path.join(MODEL_DIR, "nmg_model_tf_labels.npy"), label_encoder.classes_)

print(f"\n💾 Model saved → {MODEL_PATH}")
print(f"📏 Scaler saved → {MODEL_DIR}\\nmg_model_tf_scaler.pkl")
print(f"🏷️ Labels saved → {MODEL_DIR}\\nmg_model_tf_labels.npy")

# === Step 13: Convert to TFLite ===
print("\n⚙️ Converting to TensorFlow Lite (.tflite)...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print(f"✅ TFLite model saved → {TFLITE_PATH}")
print("\n🏁 Training + Conversion complete!")


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import pandas as pd
import tensorflow as tf
from collections import deque, Counter

# === Configuration ===
MODEL_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\Tensorflow\ASL-NonManualGrammar-Recognition-TFModels\NonManualGrammarModels"
TFLITE_MODEL_PATH = f"{MODEL_DIR}\\nmg_model_tf.tflite"
SCALER_PATH       = f"{MODEL_DIR}\\nmg_model_tf_scaler.pkl"
LABELS_PATH       = f"{MODEL_DIR}\\nmg_model_tf_labels.npy"

# === Load Model and Preprocessors ===
print("📦 Loading TFLite model and preprocessing assets...")
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

scaler = joblib.load(SCALER_PATH)
labels = np.load(LABELS_PATH, allow_pickle=True)
print(f"✅ Model loaded — expects {input_details[0]['shape'][1]} features")
print(f"🏷️ Classes: {list(labels)}")

# === MediaPipe Setup ===
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)
face_mesh = mp_face.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.8
)

# === Selected Face Landmarks (same as training) ===
SELECTED_FACE_IDXS = [
    33,133,362,263,   # eyes
    1,2,4,5,45,275,   # nose + cheeks
    61,291,0,17,57,287 # mouth + chin
]

# === Prediction Stability Settings ===
WINDOW_SIZE = 10
CONF_THRESHOLD = 0.60         # baseline prediction confidence
UNKNOWN_THRESHOLD = 0.35      # for "Can't be Found"
prediction_window = deque(maxlen=WINDOW_SIZE)

def majority_vote(predictions):
    """Returns most common prediction within the window."""
    if not predictions:
        return None
    counter = Counter(predictions)
    return counter.most_common(1)[0][0]

# === Start Camera ===
cap = cv2.VideoCapture(0)
print("📸 Camera started — press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # === Detect hands and face ===
    hand_results = hands.process(rgb)
    face_results = face_mesh.process(rgb)

    row = []
    hands_detected = False
    face_detected = False

    # --- HAND LANDMARKS (2 hands) ---
    if hand_results.multi_hand_landmarks:
        hands_detected = True
        for hand_landmarks in hand_results.multi_hand_landmarks[:2]:
            for lm in hand_landmarks.landmark:
                row.extend([lm.x, lm.y, lm.z])
        if len(hand_results.multi_hand_landmarks) == 1:
            row.extend([0] * (21 * 3))
    else:
        row.extend([0] * (21 * 3 * 2))

    # --- FACE LANDMARKS ---
    if face_results.multi_face_landmarks:
        face_detected = True
        for idx in SELECTED_FACE_IDXS:
            lm = face_results.multi_face_landmarks[0].landmark[idx]
            row.extend([lm.x, lm.y, lm.z])
    else:
        row.extend([0] * (len(SELECTED_FACE_IDXS) * 3))

    # === PREDICTION LOGIC ===
    if not (hands_detected or face_detected):
        text = "No Hands or Face to Detect"
        color = (0, 0, 255)  # red
    elif len(row) == input_details[0]['shape'][1]:
        X_live = pd.DataFrame([row])
        X_scaled = scaler.transform(X_live).astype(np.float32)

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], X_scaled)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]

        pred_conf = np.max(output)
        pred_label = labels[np.argmax(output)]

        # --- Unknown Gesture / Low Confidence ---
        if pred_conf < UNKNOWN_THRESHOLD:
            text = "Can't be Found"
            color = (0, 165, 255)  # orange
            prediction_window.clear()  # reset stability buffer
        elif pred_conf >= CONF_THRESHOLD:
            prediction_window.append(pred_label)
            stable_pred = majority_vote(prediction_window)
            text = f"{stable_pred.upper()} ({pred_conf:.2f})"
            color = (0, 255, 0)  # green
        else:
            text = f"Low Confidence ({pred_conf:.2f})"
            color = (0, 255, 255)  # yellow
    else:
        text = "⚠️ Invalid landmark vector"
        color = (0, 0, 255)

    # === Draw Landmarks ===
    if hand_results.multi_hand_landmarks:
        for hl in hand_results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hl, mp_hands.HAND_CONNECTIONS)
    if face_results.multi_face_landmarks:
        for fl in face_results.multi_face_landmarks:
            mp_draw.draw_landmarks(
                frame, fl, mp_face.FACEMESH_TESSELATION,
                mp_draw.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                mp_draw.DrawingSpec(color=(0, 0, 255), thickness=1)
            )

    # === Overlay prediction ===
    cv2.putText(frame, text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    cv2.putText(frame, f"Frames stabilized: {len(prediction_window)}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    cv2.imshow("ASL Facial + Hand Expression (TFLite)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("👋 Session ended.")